In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import os
import glob
import pandas as pd

# =========================
# CONFIG
# =========================
DATA_DIR = "D:\\MDSA Case Study\\box office data"  # folder containing yearly CSVs
OUTPUT_FILE = "merged_movies_data_1937_2025_filtered.csv"

# Expected pattern examples:
# merged_movies_data_1937.csv
# merged_movies_data_YYYY(1937-2025).csv
# merged_movies_data_1940-anything.csv
# You can tighten this pattern if your filenames are consistent.
PATTERN = os.path.join(DATA_DIR, "*.csv")

START_YEAR = 1937
END_YEAR = 2025

# Columns to drop (using your provided names)
COLUMNS_TO_DROP = [
    "Votes",
    "mÃ©ta_score",       # keep as-is from your file
    "description",
    "Movie Link",
    "writers",
    "directors",
    "stars",
    "awards_content",
]

# =========================
# HELPERS
# =========================
def normalize_colname(c: str) -> str:
    """Normalize column names for safer matching."""
    return c.strip()

def file_year_in_range(filename: str, start: int, end: int) -> bool:
    """
    Return True if filename contains at least one year between start and end.
    """
    base = os.path.basename(filename)
    # quick scan for 4-digit years
    for y in range(start, end + 1):
        if str(y) in base:
            return True
    return False

def contains_english(value) -> bool:
    """
    True if Languages contains 'English' among one or more languages.
    Handles comma/pipe/slash separated strings.
    """
    if pd.isna(value):
        return False
    s = str(value).strip().lower()
    # direct fast check first
    if "english" not in s:
        return False

    # tokenize on common separators for cleaner matching
    for sep in [",", "|", "/", ";"]:
        s = s.replace(sep, ",")
    tokens = [t.strip() for t in s.split(",") if t.strip()]
    return any(tok == "english" for tok in tokens) or ("english" in s)

def is_us_origin(value) -> bool:
    """
    True if countries_origin includes US.
    Accepts values like: US, USA, United States, United States of America
    and multi-country strings.
    """
    if pd.isna(value):
        return False
    s = str(value).strip().lower()

    # Normalize separators
    for sep in ["|", "/", ";"]:
        s = s.replace(sep, ",")
    tokens = [t.strip() for t in s.split(",") if t.strip()]

    us_aliases = {
        "us",
        "u.s.",
        "usa",
        "u.s.a.",
        "united states",
        "united states of america",
    }

    # exact token match OR fallback substring checks for messy values
    if any(tok in us_aliases for tok in tokens):
        return True

    if "united states" in s or "usa" in s or s == "us":
        return True

    return False

# =========================
# LOAD FILES
# =========================
all_files = glob.glob(PATTERN)
candidate_files = [f for f in all_files if file_year_in_range(f, START_YEAR, END_YEAR)]

if not candidate_files:
    raise FileNotFoundError(
        f"No CSV files found in {DATA_DIR} containing years {START_YEAR}-{END_YEAR}."
    )

dfs = []
for f in sorted(candidate_files):
    try:
        df = pd.read_csv(f, low_memory=False)
        # Normalize column names
        df.columns = [normalize_colname(c) for c in df.columns]
        # Track source file (optional)
        df["__source_file"] = os.path.basename(f)
        dfs.append(df)
        print(f"Loaded: {f} | rows={len(df):,}")
    except Exception as e:
        print(f"Skipped {f} due to read error: {e}")

if not dfs:
    raise RuntimeError("No files could be loaded successfully.")

merged = pd.concat(dfs, ignore_index=True, sort=False)
print(f"\nMerged rows: {len(merged):,}")

# =========================
# DROP COLUMNS
# =========================
# Drop only columns that exist (safe)
existing_drop_cols = [c for c in COLUMNS_TO_DROP if c in merged.columns]
merged = merged.drop(columns=existing_drop_cols, errors="ignore")
print(f"Dropped columns: {existing_drop_cols}")

# =========================
# FILTER CONDITIONS
# =========================
required_cols = ["Languages", "countries_origin"]
missing_required = [c for c in required_cols if c not in merged.columns]
if missing_required:
    raise KeyError(
        f"Missing required columns for filtering: {missing_required}. "
        f"Available columns: {list(merged.columns)}"
    )

filtered = merged[
    merged["Languages"].apply(contains_english) &
    merged["countries_origin"].apply(is_us_origin)
].copy()

print(f"Filtered rows (English + US origin): {len(filtered):,}")

# Optional: remove duplicates
# filtered = filtered.drop_duplicates(subset=["Title", "Year"], keep="first")

# Save output
filtered.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")
print(f"Saved: {OUTPUT_FILE}")


Loaded: D:\MDSA Case Study\box office data\merged_movies_data_1937.csv | rows=600
Loaded: D:\MDSA Case Study\box office data\merged_movies_data_1938.csv | rows=600
Loaded: D:\MDSA Case Study\box office data\merged_movies_data_1939.csv | rows=600
Loaded: D:\MDSA Case Study\box office data\merged_movies_data_1940.csv | rows=600
Loaded: D:\MDSA Case Study\box office data\merged_movies_data_1941.csv | rows=600
Loaded: D:\MDSA Case Study\box office data\merged_movies_data_1942.csv | rows=600
Loaded: D:\MDSA Case Study\box office data\merged_movies_data_1943.csv | rows=600
Loaded: D:\MDSA Case Study\box office data\merged_movies_data_1944.csv | rows=600
Loaded: D:\MDSA Case Study\box office data\merged_movies_data_1945.csv | rows=600
Loaded: D:\MDSA Case Study\box office data\merged_movies_data_1946.csv | rows=600
Loaded: D:\MDSA Case Study\box office data\merged_movies_data_1947.csv | rows=600
Loaded: D:\MDSA Case Study\box office data\merged_movies_data_1948.csv | rows=600
Loaded: D:\MDSA 

In [3]:
df = pd.read_csv(OUTPUT_FILE)

In [4]:
df

,Title,Year,Duration,MPA,Rating,méta_score,budget,opening_weekend_Gross,grossWorldWWide,gross_US_Canada,release_date,countries_origin,filming_locations,production_company,genres,Languages,__source_file
0,1. Snow White and the Seven Dwarfs,1937,1h 23m,Approved,7.6,96.0,"$1,499,000 (estimated)","$6,017,914","$184,960,747","$184,925,486","February 4, 1938",['United States'],['Walt Disney Feature Animation - 500 S. Buena...,"['Walt Disney Animation Studios', 'Walt Disney...","['Fairy Tale', 'Hand-Drawn Animation', 'Advent...",['English'],merged_movies_data_1937.csv
1,3. Stage Door,1937,1h 32m,Approved,7.7,NaN,"$952,000 (estimated)",NaN,"$8,835",NaN,"October 8, 1937",['United States'],"['RKO Studios - 780 N. Gower Street, Hollywood...",['RKO Radio Pictures'],"['Comedy', 'Drama', 'Romance']",['English'],merged_movies_data_1937.csv
2,4. Shall We Dance,1937,1h 49m,Approved,7.4,NaN,"$991,000 (estimated)",NaN,"$6,662",NaN,"May 7, 1937",['United States'],"['RKO Studios - 780 N. Gower Street, Hollywood...",['RKO Radio Pictures'],"['Romantic Comedy', 'Comedy', 'Musical', 'Roma...","['English', 'French']",merged_movies_data_1937.csv
3,5. The Great Garrick,1937,1h 29m,Approved,6.7,NaN,NaN,NaN,NaN,NaN,"October 30, 1937",['United States'],['Warner Brothers Burbank Studios - 4000 Warne...,['Warner Bros.'],"['Comedy', 'Romance']","['English', 'French']",merged_movies_data_1937.csv
4,6. Lost Horizon,1937,2h 12m,Approved,7.6,NaN,"$4,000,000 (estimated)",NaN,NaN,NaN,"September 1, 1937",['United States'],"['Ojai, California, USA']",['Columbia Pictures'],"['Adventure', 'Drama', 'Fantasy', 'Mystery']","['English', 'Mandarin']",merged_movies_data_1937.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28355,591. The Street Avenger,2025,NaN,NaN,NaN,NaN,"$4,000,000 (estimated)",NaN,NaN,NaN,"December 24, 2025",['United States'],"['New Jersey, USA (Studio)']",['Angry Troll Studio'],"['Action', 'Adventure', 'Crime', 'Thriller']",['English'],merged_movies_data_2025.csv
28356,594. Friday the 13th: Blood Loss,2025,1h 30m,NaN,5.4,NaN,"$30,000 (estimated)",NaN,NaN,NaN,"March 16, 2025",['United States'],"['Rochester New York, USA (location)']",['Forever Entertainment'],"['Drama', 'Horror']",['English'],merged_movies_data_2025.csv
28357,595. Killer Clown Girls,2025,1h 30m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"December 30, 2025",['United States'],NaN,"['Masch Global Media', 'Masch', 'Roman Empire ...",['Horror'],['English'],merged_movies_data_2025.csv
28358,"597. Love, Brooklyn",2025,1h 37m,NaN,6.9,60.0,NaN,NaN,NaN,NaN,"January 27, 2025",['United States'],"['Brooklyn, New York, USA']","['Daughter Films', 'Fireheart Entertainment', ...",['Drama'],['English'],merged_movies_data_2025.csv


In [5]:
horror_data = df[
    (df['genres'].str.contains(r'Horror', case=False, na=False, regex=True)) 
].copy()

In [6]:
horror_data.head()

,Title,Year,Duration,MPA,Rating,méta_score,budget,opening_weekend_Gross,grossWorldWWide,gross_US_Canada,release_date,countries_origin,filming_locations,production_company,genres,Languages,__source_file
211,290. SOS Coast Guard,1937,3h 44m,Approved,5.9,NaN,"$107,217 (estimated)",NaN,NaN,NaN,"August 28, 1937",['United States'],"['Port Hueneme, California, USA']",['Republic Pictures (I)'],"['Adventure', 'Horror', 'Romance', 'Sci-Fi']",['English'],merged_movies_data_1937.csv
502,128. The Black Doll,1938,1h 6m,Approved,5.6,NaN,NaN,NaN,NaN,NaN,"January 30, 1938",['United States'],['Universal Studios - 100 Universal City Plaza...,['Universal Pictures'],"['Comedy', 'Crime', 'Horror', 'Mystery', 'Roma...","['English', 'Spanish']",merged_movies_data_1938.csv
579,223. The Patient in Room 18,1938,59m,Approved,5.8,NaN,NaN,NaN,NaN,NaN,"January 8, 1938",['United States'],['Warner Brothers Burbank Studios - 4000 Warne...,['Warner Bros.'],"['Comedy', 'Crime', 'Horror', 'Mystery', 'Roma...","['English', 'French']",merged_movies_data_1938.csv
804,15. Son of Frankenstein,1939,1h 39m,Approved,7.1,NaN,NaN,NaN,$127,NaN,"January 13, 1939",['United States'],['Universal Studios - 100 Universal City Plaza...,['Universal Pictures'],"['Monster Horror', 'Drama', 'Horror', 'Sci-Fi']",['English'],merged_movies_data_1939.csv
833,48. The Cat and the Canary,1939,1h 12m,Approved,7.1,NaN,NaN,NaN,NaN,NaN,"November 10, 1939",['United States'],"['Paramount Studios - 5555 Melrose Avenue, Hol...",['Paramount Pictures'],"['Comedy', 'Horror', 'Mystery', 'Thriller']",['English'],merged_movies_data_1939.csv


In [7]:
horror_data.drop(columns=['__source_file', 'méta_score', 'countries_origin'], inplace=True, errors='ignore')

In [8]:
horror_data = horror_data[horror_data["opening_weekend_Gross"].notna()]

In [9]:
horror_data.head()

,Title,Year,Duration,MPA,Rating,budget,opening_weekend_Gross,grossWorldWWide,gross_US_Canada,release_date,filming_locations,production_company,genres,Languages
9372,13. Night of the Living Dead,1968,1h 36m,R,7.8,"$114,000 (estimated)","$5,452","$237,994","$236,452","October 4, 1968","['Evans City Cemetery, Evans City, Pennsylvani...",['Image Ten'],"['B-Horror', 'Supernatural Horror', 'Survival'...",['English']
9834,19. Multiple Maniacs,1970,1h 31m,Not Rated,6.5,"$5,000 (estimated)","$9,814","$39,025","$33,036","January 1, 2022","['Baltimore, Maryland, USA']",['Dreamland'],"['Dark Comedy', 'Satire', 'Comedy', 'Crime', '...",['English']
10527,1. The Exorcist,1973,2h 2m,R,8.1,"$11,000,000 (estimated)","$8,175,666","$430,872,776","$233,005,644","December 26, 1973","['Mosul, Iraq']","['Warner Bros.', 'Hoya Productions']","['Body Horror', 'Supernatural Horror', 'Horror']","['English', 'Latin', 'Greek', 'French', 'Germa..."
10540,20. Ganja & Hess,1973,1h 52m,R,6.2,NaN,"$3,406","$21,197","$21,197","March 18, 1988","['Croton-on-Hudson, New York, USA']",['Kelly/Jordan Enterprises'],"['Dark Fantasy', 'Dark Romance', 'Drama', 'Fan...","['English', 'French']"
10971,1. Jaws,1975,2h 4m,PG,8.1,"$7,000,000 (estimated)","$7,061,513","$477,916,625","$267,263,625","June 20, 1975","[""Water Street, Edgartown, Martha's Vineyard, ...","['Zanuck/Brown Productions', 'Universal Pictur...","['Monster Horror', 'Sea Adventure', 'Survival'...",['English']


In [10]:
horror_data["Title"] = (
    horror_data["Title"]
    .fillna("")
    .astype(str)
    .str.strip()
    .str.replace(r'^\s*\d+\s*[\.\)\-:]\s*', '', regex=True)
)


In [11]:
horror_data.head()

,Title,Year,Duration,MPA,Rating,budget,opening_weekend_Gross,grossWorldWWide,gross_US_Canada,release_date,filming_locations,production_company,genres,Languages
9372,Night of the Living Dead,1968,1h 36m,R,7.8,"$114,000 (estimated)","$5,452","$237,994","$236,452","October 4, 1968","['Evans City Cemetery, Evans City, Pennsylvani...",['Image Ten'],"['B-Horror', 'Supernatural Horror', 'Survival'...",['English']
9834,Multiple Maniacs,1970,1h 31m,Not Rated,6.5,"$5,000 (estimated)","$9,814","$39,025","$33,036","January 1, 2022","['Baltimore, Maryland, USA']",['Dreamland'],"['Dark Comedy', 'Satire', 'Comedy', 'Crime', '...",['English']
10527,The Exorcist,1973,2h 2m,R,8.1,"$11,000,000 (estimated)","$8,175,666","$430,872,776","$233,005,644","December 26, 1973","['Mosul, Iraq']","['Warner Bros.', 'Hoya Productions']","['Body Horror', 'Supernatural Horror', 'Horror']","['English', 'Latin', 'Greek', 'French', 'Germa..."
10540,Ganja & Hess,1973,1h 52m,R,6.2,NaN,"$3,406","$21,197","$21,197","March 18, 1988","['Croton-on-Hudson, New York, USA']",['Kelly/Jordan Enterprises'],"['Dark Fantasy', 'Dark Romance', 'Drama', 'Fan...","['English', 'French']"
10971,Jaws,1975,2h 4m,PG,8.1,"$7,000,000 (estimated)","$7,061,513","$477,916,625","$267,263,625","June 20, 1975","[""Water Street, Edgartown, Martha's Vineyard, ...","['Zanuck/Brown Productions', 'Universal Pictur...","['Monster Horror', 'Sea Adventure', 'Survival'...",['English']


In [13]:
horror_data.drop(columns=['Languages', 'genres', 'production_company'], inplace=True, errors='ignore')

In [14]:
horror_data["filming_locations"].value_counts()

filming_locations
['Los Angeles, California, USA']                                                               35
['Vancouver, British Columbia, Canada']                                                        21
['Toronto, Ontario, Canada']                                                                   17
['New Orleans, Louisiana, USA']                                                                15
['Santa Clarita, California, USA']                                                             10
                                                                                               ..
['Raleigh Studios - 5300 Melrose Avenue, Hollywood, Los Angeles, California, USA (Studio)']     1
['Effingham County, Georgia, USA']                                                              1
['Vianden, Luxembourg (Castle interior scenes)']                                                1
['Azusa, California, USA (location)']                                                           1
['

In [15]:
# 1) Convert release_date like "August 3, 1938" -> datetime -> "1938-08-03"
horror_data["release_date"] = pd.to_datetime(
    horror_data["release_date"],
    format="%B %d, %Y",   # e.g., August 3, 1938
    errors="coerce"       # invalid dates become NaT
)

# If you want the column stored as text in YYYY-MM-DD format:
horror_data["release_date"] = horror_data["release_date"].dt.strftime("%Y-%m-%d")

# Convert back to datetime for month extraction (safe/easy for season logic)
horror_data["release_date"] = pd.to_datetime(horror_data["release_date"], errors="coerce")

# 2) Create seasons from month
month_to_season = {
    12: "Winter", 1: "Winter", 2: "Winter",
    3: "Spring", 4: "Spring", 5: "Spring",
    6: "Summer", 7: "Summer", 8: "Summer",
    9: "Fall", 10: "Fall", 11: "Fall"
}

horror_data["seasons"] = horror_data["release_date"].dt.month.map(month_to_season)

# Optional: if date is missing/invalid, label as Unknown
horror_data["seasons"] = horror_data["seasons"].fillna("Unknown")


In [16]:
horror_data["release_month"] = horror_data["release_date"].dt.month
horror_data["release_month"] = horror_data["release_month"].fillna(0).astype(int)

In [17]:
import re
import ast
import pandas as pd
import pycountry

# Build lookup maps once
name_to_iso3 = {}
for c in pycountry.countries:
    # official/common names
    name_to_iso3[c.name.lower()] = c.alpha_3
    if hasattr(c, "official_name"):
        name_to_iso3[c.official_name.lower()] = c.alpha_3

# common aliases
ALIASES = {
    "usa": "USA", "u.s.a": "USA", "u.s.": "USA", "us": "USA", "united states": "USA",
    "uk": "GBR", "u.k.": "GBR", "united kingdom": "GBR", "england": "GBR",
    "south korea": "KOR", "north korea": "PRK", "puerto rico": "PRI",
    "russia": "RUS", "vietnam": "VNM", "italy": "ITA", "france": "FRA", "germany": "DEU", "spain": "ESP", "jordan": "JOR", "lebanon": "LBN", "egypt": "EGY", "iran": "IRN", "iraq": "IRQ", "syria": "SYR",
    "india": "IND", "china": "CHN", "japan": "JPN", "canada": "CAN", "australia": "AUS", "brazil": "BRA", "mexico": "MEX", "argentina": "ARG", "south africa": "ZAF"
}
name_to_iso3.update(ALIASES)

def normalize_text(x):
    if pd.isna(x): return ""
    s = str(x).strip()
    if s.startswith("[") and s.endswith("]"):
        try:
            parsed = ast.literal_eval(s)
            if isinstance(parsed, list):
                s = " ; ".join(str(i) for i in parsed)
        except Exception:
            pass
    s = re.sub(r"\([^)]*\)", " ", s)      # remove (studio), (location), dates
    s = re.sub(r"[\n\r]+", " ; ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def extract_countries_broad(value):
    s = normalize_text(value).lower()
    if not s:
        return []

    found = []
    # phrase-level scan against known country names
    for name, iso3 in name_to_iso3.items():
        if re.search(rf"\b{re.escape(name)}\b", s):
            found.append(iso3)

    # dedupe keep order
    out, seen = [], set()
    for c in found:
        if c not in seen:
            seen.add(c)
            out.append(c)

    return out

horror_data["filming_countries_list"] = horror_data["filming_locations"].apply(extract_countries_broad)
horror_data["filming_countries"] = horror_data["filming_countries_list"].apply(lambda x: ", ".join(x) if x else "Unknown")


In [18]:
def usa_only_if_present(x):
    if pd.isna(x):
        return "Unknown"
    parts = [p.strip().upper() for p in re.split(r"[,\|;/]+", str(x)) if p.strip()]
    if any(p in {"USA","US","U.S.","U.S.A.","UNITED STATES"} for p in parts):
        return "USA"
    return str(x).strip() if str(x).strip() else "Unknown"

horror_data["filming_countries"] = horror_data["filming_countries"].apply(usa_only_if_present)

In [19]:
# pip install pycountry pycountry-convert
import re
import pandas as pd
import pycountry
import pycountry_convert as pc

# -----------------------------
# 1) Continent mapping setup
# -----------------------------
CONTINENT_NAME = {
    "AF": "Africa",
    "AS": "Asia",
    "EU": "Europe",
    "NA": "North America",
    "SA": "South America",
    "OC": "Oceania",
    "AN": "Antarctica"
}

ISO3_OVERRIDES = {
    "XKX": "Europe",         # Kosovo
    "PRI": "North America",  # Puerto Rico
    "RUS": "Europe"          # choose Europe for your business logic
}

def iso3_to_continent(iso3):
    if pd.isna(iso3):
        return "Unknown"

    iso3 = str(iso3).strip().upper()
    if not iso3:
        return "Unknown"

    if iso3 in ISO3_OVERRIDES:
        return ISO3_OVERRIDES[iso3]

    try:
        country = pycountry.countries.get(alpha_3=iso3)
        if not country:
            return "Unknown"
        iso2 = country.alpha_2
        cont_code = pc.country_alpha2_to_continent_code(iso2)
        return CONTINENT_NAME.get(cont_code, "Unknown")
    except Exception:
        return "Unknown"

# -----------------------------
# 2) Parse filming_countries -> ISO3 list
# -----------------------------
def split_iso3_list(x):
    """
    Converts strings like:
      'USA, CAN, GBR'
      'USA|CAN'
      'USA; CAN'
    to ['USA','CAN','GBR'].
    """
    if pd.isna(x):
        return []

    parts = [p.strip().upper() for p in re.split(r"[,\|;/]+", str(x)) if p.strip()]
    # keep only 3-letter codes
    parts = [p for p in parts if re.fullmatch(r"[A-Z]{3}", p)]

    # dedupe preserve order
    seen = set()
    out = []
    for p in parts:
        if p not in seen:
            seen.add(p)
            out.append(p)
    return out

# Make sure this column exists in your dataframe
# horror_data["filming_countries"] should already contain ISO3-like codes e.g. "USA, CAN"
horror_data["country_iso3_list"] = horror_data["filming_countries"].apply(split_iso3_list)

# Primary country and continent
horror_data["country_iso3_primary"] = horror_data["country_iso3_list"].apply(
    lambda x: x[0] if x else "UNK"
)
horror_data["continents"] = horror_data["country_iso3_primary"].apply(iso3_to_continent)

In [20]:
horror_data.columns

Index(['Title', 'Year', 'Duration', 'MPA', 'Rating', 'budget',
       'opening_weekend_Gross', 'grossWorldWWide', 'gross_US_Canada',
       'release_date', 'filming_locations', 'seasons', 'release_month',
       'filming_countries_list', 'filming_countries', 'country_iso3_list',
       'country_iso3_primary', 'continents'],
      dtype='object')

In [21]:
horror_data.drop(columns=['filming_countries_list', 'country_iso3_list', 'filming_countries', 'filming_locations'], inplace=True, errors='ignore')

In [22]:
horror_data.rename(columns={"country_iso3_primary": "filming_country"}, inplace=True)

In [ ]:
import numpy as np
import pandas as pd

money_cols = ["budget", "opening_weekend_Gross", "grossWorldWWide", "gross_US_Canada"]

def clean_money_col(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.strip()

    # Standardize missing markers
    s = s.replace(
        ["", "Missing value", "missing value", "N/A", "n/a", "None", "null", "-", "--"],
        np.nan
    )

    # Remove "(estimated)" tag (case-insensitive)
    s = s.str.replace(r"\(estimated\)", "", regex=True, case=False)

    # Remove currency symbols, commas, spaces and any non-numeric except dot/minus
    s = s.str.replace(r"[^0-9.\-]", "", regex=True)

    # Empty leftovers -> NaN
    s = s.replace("", np.nan)

    # Convert
    return pd.to_numeric(s, errors="coerce")

for col in money_cols:
    horror_data[col] = clean_money_col(horror_data[col])

# 1) Profit = worldwide gross - budget
horror_data["profit"] = horror_data["grossWorldWWide"] - horror_data["budget"]

# 2) ROI = (profit / budget)
horror_data["roi"] = np.where(
    horror_data["budget"] > 0,
    horror_data["profit"] / horror_data["budget"],
    np.nan
)

# 3) Domestic share = domestic gross / worldwide gross
horror_data["domestic_share"] = np.where(
    horror_data["grossWorldWWide"] > 0,
    horror_data["gross_US_Canada"] / horror_data["grossWorldWWide"],
    np.nan
)

# 4) International gross share = (worldwide - domestic) / worldwide
# clip(lower=0) prevents tiny negative values from dirty data
intl_gross = (horror_data["grossWorldWWide"] - horror_data["gross_US_Canada"]).clip(lower=0)

horror_data["intl_gross_share"] = np.where(
    horror_data["grossWorldWWide"] > 0,
    intl_gross / horror_data["grossWorldWWide"],
    np.nan
)


horror_data["roi"] = horror_data["roi"] * 100
horror_data["domestic_share"] = horror_data["domestic_share"] * 100
horror_data["intl_gross_share"] = horror_data["intl_gross_share"] * 100

In [24]:
horror_data.to_csv("horror_movies_data.csv", index=False, encoding="utf-8-sig")

In [25]:
genre_mpaa_summary = (horror_data.groupby(['release_month', 'seasons'], as_index=False)
      .agg(
          total_gross=('grossWorldWWide', 'sum'),
          avg_gross=('grossWorldWWide', 'mean'),
          movie_count=('grossWorldWWide', 'count')
      )
      .sort_values('total_gross', ascending=False)
)

In [26]:
genre_mpaa_summary.head()

,release_month,seasons,total_gross,avg_gross,movie_count
10,10,Fall,7.105200e+09,4.866575e+07,146
8,8,Summer,6.540779e+09,5.405602e+07,121
9,9,Fall,5.371132e+09,4.711520e+07,114
6,6,Summer,5.218133e+09,6.957510e+07,75
7,7,Summer,4.315601e+09,5.532821e+07,78


In [27]:
country_hits_summary = (
    horror_data.groupby('filming_country', as_index=False)
      .agg(
          profitable_movies=('profit', lambda s: (s > 0).sum()),
          roi_positive_movies=('roi', lambda s: (s > 0).sum()),
          movie_count=('profit', 'count'),
          avg_roi=('roi', 'mean'),
          total_profit=('profit', 'sum')
      )
      .sort_values(['roi_positive_movies', 'profitable_movies'], ascending=False)
)

In [65]:
min_movies = 3
genre_mpaa_summary_asc = genre_mpaa_summary[genre_mpaa_summary['movie_count'] >= min_movies] \
    .sort_values('total_gross', ascending=True)

genre_mpaa_summary_asc

,genres,MPA,total_gross,avg_gross,movie_count
15,Documentary,PG,1976133,3.952266e+05,5
14,Documentary,G,117136007,2.928400e+07,4
18,Drama,PG,359679752,3.269816e+07,11
11,Comedy,G,378146755,6.302446e+07,6
4,Adventure,G,546397786,4.967253e+07,11
7,Animation,Approved,550729761,1.101460e+08,5
12,Comedy,PG,1411074809,5.039553e+07,28
3,Action,PG-13,1906970750,1.906971e+08,10
2,Action,PG,1977098789,7.908395e+07,25
5,Adventure,PG,4318134551,9.813942e+07,44


In [34]:
streaming_data = pd.read_csv(r'D:\MDSA Case Study\paid_video_streaming_services.csv')

In [68]:
streaming_data.head()

,service_name,type,category,countries_available,monthly_price_usd,annual_price_usd,launch_year,subscribers_millions,content_type,platforms,...,device_gaming_console_pct,device_other_pct,engagement_cluster,arpu_usd,churn_rate_pct,subscribers_2020_millions,subscribers_2021_millions,subscribers_2022_millions,subscribers_2023_millions,subscribers_2024_millions
0,Netflix,VOD,Paid,Global (190+ countries),15.99,191.88,1997,301.6,"Movies, TV Shows, Originals","Web, Mobile, Smart TV, Gaming Consoles",...,9.1,6.1,High Engagement,17.80,1.68,221.7,239.4,258.6,279.3,301.6
1,Amazon Prime Video,VOD,Paid,Global (200+ countries),8.99,107.88,2006,220.0,"Movies, TV Shows, Originals","Web, Mobile, Smart TV, Gaming Consoles",...,6.2,3.1,High Engagement,8.30,1.68,161.7,174.6,188.6,203.7,220.0
2,Disney+,VOD,Paid,Global (100+ countries),10.99,109.99,2019,131.6,"Movies, TV Shows, Disney Content","Web, Mobile, Smart TV, Gaming Consoles",...,6.7,7.6,High Engagement,11.16,1.89,53.9,67.4,84.2,105.3,131.6
3,Max (HBO Max),VOD,Paid,"US, Latin America, Europe, Asia",15.99,149.99,2020,128.0,"Movies, TV Shows, HBO Originals","Web, Mobile, Smart TV, Gaming Consoles",...,8.6,5.5,High Engagement,14.51,2.10,12.8,65.5,81.9,102.4,128.0
4,iQiyi,Regional,Paid,"China, Southeast Asia, Global",2.99,29.99,2010,125.0,"Movies, TV Shows, Originals","Web, Mobile, Smart TV",...,9.2,2.5,High Engagement,2.63,1.68,91.9,99.2,107.2,115.7,125.0


In [35]:
streaming_data = streaming_data[streaming_data["service_name"].isin(['Netflix', 'Hulu', 'Amazon Prime Video', 'Disney+', 'Apple TV+', 'Max (HBO Max)'])]

In [36]:
streaming_data.head()

,service_name,type,category,countries_available,monthly_price_usd,annual_price_usd,launch_year,subscribers_millions,content_type,platforms,...,device_gaming_console_pct,device_other_pct,engagement_cluster,arpu_usd,churn_rate_pct,subscribers_2020_millions,subscribers_2021_millions,subscribers_2022_millions,subscribers_2023_millions,subscribers_2024_millions
0,Netflix,VOD,Paid,Global (190+ countries),15.99,191.88,1997,301.6,"Movies, TV Shows, Originals","Web, Mobile, Smart TV, Gaming Consoles",...,9.1,6.1,High Engagement,17.80,1.68,221.7,239.4,258.6,279.3,301.6
1,Amazon Prime Video,VOD,Paid,Global (200+ countries),8.99,107.88,2006,220.0,"Movies, TV Shows, Originals","Web, Mobile, Smart TV, Gaming Consoles",...,6.2,3.1,High Engagement,8.30,1.68,161.7,174.6,188.6,203.7,220.0
2,Disney+,VOD,Paid,Global (100+ countries),10.99,109.99,2019,131.6,"Movies, TV Shows, Disney Content","Web, Mobile, Smart TV, Gaming Consoles",...,6.7,7.6,High Engagement,11.16,1.89,53.9,67.4,84.2,105.3,131.6
3,Max (HBO Max),VOD,Paid,"US, Latin America, Europe, Asia",15.99,149.99,2020,128.0,"Movies, TV Shows, HBO Originals","Web, Mobile, Smart TV, Gaming Consoles",...,8.6,5.5,High Engagement,14.51,2.10,12.8,65.5,81.9,102.4,128.0
12,Hulu,VOD,Paid,"US, Japan",7.99,79.99,2007,50.2,"Movies, TV Shows, Originals","Web, Mobile, Smart TV, Gaming Consoles",...,2.5,2.4,Medium Engagement,8.10,2.04,36.9,39.9,43.0,46.5,50.2


In [71]:
streaming_data.columns

Index(['service_name', 'type', 'category', 'countries_available',
       'monthly_price_usd', 'annual_price_usd', 'launch_year',
       'subscribers_millions', 'content_type', 'platforms', 'is_free',
       'parent_company', 'age_group_18_24_pct', 'age_group_25_34_pct',
       'age_group_35_44_pct', 'age_group_45_54_pct', 'age_group_55_64_pct',
       'age_group_65_plus_pct', 'device_android_pct', 'device_ios_pct',
       'device_web_pct', 'device_smart_tv_pct', 'device_gaming_console_pct',
       'device_other_pct', 'engagement_cluster', 'arpu_usd', 'churn_rate_pct',
       'subscribers_2020_millions', 'subscribers_2021_millions',
       'subscribers_2022_millions', 'subscribers_2023_millions',
       'subscribers_2024_millions'],
      dtype='object')

In [37]:
streaming_data.drop(columns=['countries_available', 'type', 'launch_year', 'content_type', 'platforms', 'is_free', 'parent_company' ], inplace=True, errors='ignore')

In [38]:
streaming_data.drop(columns=['category'], inplace=True, errors='ignore')

In [39]:
streaming_data.columns

Index(['service_name', 'monthly_price_usd', 'annual_price_usd',
       'subscribers_millions', 'age_group_18_24_pct', 'age_group_25_34_pct',
       'age_group_35_44_pct', 'age_group_45_54_pct', 'age_group_55_64_pct',
       'age_group_65_plus_pct', 'device_android_pct', 'device_ios_pct',
       'device_web_pct', 'device_smart_tv_pct', 'device_gaming_console_pct',
       'device_other_pct', 'engagement_cluster', 'arpu_usd', 'churn_rate_pct',
       'subscribers_2020_millions', 'subscribers_2021_millions',
       'subscribers_2022_millions', 'subscribers_2023_millions',
       'subscribers_2024_millions'],
      dtype='object')

In [40]:
streaming_data.rename(columns={'subscribers_millions': 'total_subscribers(till 2024)'}, inplace=True)

In [41]:
import pandas as pd
import re

df1 = streaming_data

year_cols = [
    "subscribers_2020_millions",
    "subscribers_2021_millions",
    "subscribers_2022_millions",
    "subscribers_2023_millions",
    "subscribers_2024_millions",
]

# Keep everything else as id vars
id_cols = [c for c in df1.columns if c not in year_cols]

# Melt only yearly subscriber columns
subs_long = df1.melt(
    id_vars=id_cols,
    value_vars=year_cols,
    var_name="year_col",
    value_name="subscribers_millions"
)

# Extract numeric year
subs_long["year"] = subs_long["year_col"].str.extract(r"(\d{4})").astype(int)
subs_long = subs_long.drop(columns=["year_col"])


subs_long["subscribers_millions"] = pd.to_numeric(subs_long["subscribers_millions"], errors="coerce")

subs_long = subs_long.sort_values(["service_name", "year"]).reset_index(drop=True)


subs_long["yoy_growth_pct"] = (
    subs_long.groupby("service_name")["subscribers_millions"]
    .pct_change()
    .mul(100)
)

subs_long["yoy_growth_millions"] = (
    subs_long.groupby("service_name")["subscribers_millions"]
    .diff()
)

In [42]:
subs_long.head()

,service_name,monthly_price_usd,annual_price_usd,total_subscribers(till 2024),age_group_18_24_pct,age_group_25_34_pct,age_group_35_44_pct,age_group_45_54_pct,age_group_55_64_pct,age_group_65_plus_pct,...,device_smart_tv_pct,device_gaming_console_pct,device_other_pct,engagement_cluster,arpu_usd,churn_rate_pct,subscribers_millions,year,yoy_growth_pct,yoy_growth_millions
0,Amazon Prime Video,8.99,107.88,220.0,8.8,32.6,19.0,13.1,10.4,9.3,...,13.3,6.2,3.1,High Engagement,8.3,1.68,161.7,2020,NaN,NaN
1,Amazon Prime Video,8.99,107.88,220.0,8.8,32.6,19.0,13.1,10.4,9.3,...,13.3,6.2,3.1,High Engagement,8.3,1.68,174.6,2021,7.977737,12.9
2,Amazon Prime Video,8.99,107.88,220.0,8.8,32.6,19.0,13.1,10.4,9.3,...,13.3,6.2,3.1,High Engagement,8.3,1.68,188.6,2022,8.018328,14.0
3,Amazon Prime Video,8.99,107.88,220.0,8.8,32.6,19.0,13.1,10.4,9.3,...,13.3,6.2,3.1,High Engagement,8.3,1.68,203.7,2023,8.006363,15.1
4,Amazon Prime Video,8.99,107.88,220.0,8.8,32.6,19.0,13.1,10.4,9.3,...,13.3,6.2,3.1,High Engagement,8.3,1.68,220.0,2024,8.001964,16.3


In [43]:
subs_long.to_csv("streaming_subscribers_data.csv", index=False, encoding="utf-8-sig")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [3]:
all_movie = pd.read_csv(r'D:\MDSA Case Study\Movies Daily Update Dataset export 2026-02-16 00-40-22\Movies Daily Update Dataset export 2026-02-16 00-40-22.csv')

In [4]:
all_movie.drop(columns=['credits', 'keywords', 'poster_path', 'backdrop_path', 'recommendations', 'tagline', 'overview', 'id'], inplace=True)

In [5]:
# Keep only rows where original_language == 'en'
all_movie = all_movie[all_movie['original_language'] == 'en'].copy()

In [6]:
all_movie = all_movie[all_movie['status'] == 'Released']

In [7]:
all_movie.drop(columns=['original_language', 'status', 'popularity'], inplace=True, errors='ignore')

In [8]:
all_movie = all_movie[all_movie['release_date'] >= '1937-01-01'].copy()

In [9]:
all_movie = all_movie[all_movie['revenue'] >= 10000].copy()

In [10]:
to_add = all_movie[
    (all_movie['production_companies'].str.contains(r'Disney', case=False, na=False, regex=True)) &
    (pd.to_datetime(all_movie['release_date'], errors='coerce') >= pd.Timestamp('2017-01-01'))
].copy()

In [11]:
to_add.drop(columns=['vote_average', 'vote_count', 'budget', 'runtime'], inplace=True)

In [12]:
to_add.rename(columns={
    'title': 'Movie Title',
    'release_date': 'Date Released',
    'genres': 'Genre',
}, inplace=True)

In [14]:
to_add.drop(columns=['production_companies'], inplace=True, errors='ignore')

In [15]:
to_add.rename(columns={
    'revenue': 'Inflation Adjusted Gross'}, inplace=True)

In [13]:
disney_data = pd.read_csv(r'D:\MDSA Case Study\disney movie total gross.csv')
disney_data.head()

,Movie Title,Date Released,Genre,MPAA Rating,Total Gross,Inflation Adjusted Gross
0,Snow White and the Seven Dwarfs,21/12/1937,Musical,G,184925485,5228953251
1,Pinocchio,09/02/1940,Adventure,G,84300000,2188229052
2,Fantasia,13/11/1940,Musical,G,83320000,2187090808
3,Song of the South,12/11/1946,Adventure,G,65000000,1078510579
4,Cinderella,15/02/1950,Drama,G,85000000,920608730


In [16]:
combined = pd.concat([disney_data, to_add], ignore_index=True)

In [17]:
combined.drop(columns=['Total Gross'], inplace=True)

In [18]:
allowed = {
    "Comedy","Adventure","Drama","Action","Thriller/Suspense","Romantic Comedy",
    "Unknown","Musical","Documentary","Western","Horror","Black Comedy","Concert/Performance"
}

def first_allowed_genre(s):
    if pd.isna(s):
        return "Unknown"
    parts = [p.strip() for p in re.split(r'[-/,|]', str(s))]
    for p in parts:
        if p in allowed:
            return p
    return "Unknown"

combined['Genre'] = combined['Genre'].apply(first_allowed_genre)

In [19]:
combined.loc[
    combined['Movie Title'].str.contains(r'beauty\s+and\s+the\s+beast', case=False, na=False),
    'Genre'
] = 'Musical'

In [20]:
date_col = 'Date Released'

# String version for pattern checks
s = combined[date_col].astype(str).str.strip()

# Only pre-2017-style rows (DD/MM/YYYY) -> parse and reformat
mask_pre = s.str.contains('/', na=False)

combined.loc[mask_pre, date_col] = pd.to_datetime(
    combined.loc[mask_pre, date_col],
    format='%d/%m/%Y',
    errors='coerce'
).dt.strftime('%Y-%m-%d')

In [21]:
mpaa_map = {
    "Haunted Mansion": "PG-13",
    "Strange World": "PG",
    "Lightyear": "PG",
    "Turning Red": "PG",
    "Encanto": "PG",
    "Jungle Cruise": "PG-13",
    "Luca": "PG",
    "Cruella": "PG-13",
    "Raya and the Last Dragon": "PG",
    "Soul": "PG",
    "Mulan": "PG-13",
    "Onward": "PG",
    "Frozen II": "PG",
    "Maleficent: Mistress of Evil": "PG",
    "The Lion King": "PG",
    "Toy Story 4": "G",
    "Aladdin": "PG",
    "Dumbo": "PG",
    "Mary Poppins Returns": "PG",
    "Ralph Breaks the Internet": "PG",
    "The Nutcracker and the Four Realms": "PG",
    "Christopher Robin": "PG",
    "Incredibles 2": "PG",
    "Solo: A Star Wars Story": "PG-13",
    "A Wrinkle in Time": "PG",
    "Coco": "PG",
    "Pirates of the Caribbean: Dead Men Tell No Tales": "PG-13",
    "Beauty and the Beast": "PG"
}

# Fill only missing MPAA Rating values based on Movie Title
missing_mask = combined["MPAA Rating"].isna() | (combined["MPAA Rating"].astype(str).str.strip() == "")
combined.loc[missing_mask, "MPAA Rating"] = combined.loc[missing_mask, "Movie Title"].map(mpaa_map)

In [25]:

df = combined.copy()

DATE_COL  = "Date Released"
TITLE_COL = "Movie Title"
GENRE_COL = "Genre"
MPAA_COL  = "MPAA Rating"
GROSS_COL = "Inflation Adjusted Gross"

# 1) Ensure date is datetime
df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")

# 2) Basic time dimensions (super useful for dashboards)
df["release_year"]    = df[DATE_COL].dt.year
df["release_month"]   = df[DATE_COL].dt.month
df["release_quarter"] = df[DATE_COL].dt.to_period("Q").astype(str)
df["release_dow"]     = df[DATE_COL].dt.day_name()

# 3) Seasons (meteorological)
# Winter: Dec-Feb | Spring: Mar-May | Summer: Jun-Aug | Fall: Sep-Nov
season_map = {
    12: "Winter", 1: "Winter", 2: "Winter",
    3: "Spring", 4: "Spring", 5: "Spring",
    6: "Summer", 7: "Summer", 8: "Summer",
    9: "Fall", 10: "Fall", 11: "Fall",
}
df["season"] = df["release_month"].map(season_map)

# Optional: special "Holiday" tag (common in studio strategy)
df["is_holiday_window"] = df["release_month"].isin([11, 12]).astype(int)

# 4) Gross metrics (great for charts)
df["gross_millions"] = df[GROSS_COL] / 1_000_000

# 6) Performance ranks (useful for “Top movies by year/decade/genre” visuals)
df["rank_gross_all_time"] = df[GROSS_COL].rank(method="dense", ascending=False).astype(int)

df["rank_gross_by_year"] = (
    df.groupby("release_year")[GROSS_COL]
      .rank(method="dense", ascending=False)
      .astype(int)
)

df["rank_gross_by_genre"] = (
    df.groupby(GENRE_COL)[GROSS_COL]
      .rank(method="dense", ascending=False)
      .astype(int)
)

# 8) MPAA simplification (optional but good for dashboard filters)
# Keeps only core buckets; everything else -> "Other/Unknown"
core_mpaa = {"G", "PG", "PG-13", "R", "Not Rated", "Unknown"}
df["mpaa_bucket"] = df[MPAA_COL].where(df[MPAA_COL].isin(core_mpaa), "Other/Unknown")

# 10) Cumulative gross over time (nice story chart)
df["cumulative_gross"] = df[GROSS_COL].cumsum()
df["cumulative_gross_m"] = df["cumulative_gross"] / 1_000_000

# 11) Shares (genre and MPAA share of total gross)
total_gross = df[GROSS_COL].sum()
df["share_of_total_gross"] = df[GROSS_COL] / total_gross

df["genre_total_gross"] = df.groupby(GENRE_COL)[GROSS_COL].transform("sum")
df["genre_share_of_total"] = df["genre_total_gross"] / total_gross

df["mpaa_total_gross"] = df.groupby("mpaa_bucket")[GROSS_COL].transform("sum")
df["mpaa_share_of_total"] = df["mpaa_total_gross"] / total_gross

# Done
df.head()

,Movie Title,Date Released,Genre,MPAA Rating,Inflation Adjusted Gross,release_year,release_month,release_quarter,release_dow,season,...,rank_gross_by_year,rank_gross_by_genre,mpaa_bucket,cumulative_gross,cumulative_gross_m,share_of_total_gross,genre_total_gross,genre_share_of_total,mpaa_total_gross,mpaa_share_of_total
0,Snow White and the Seven Dwarfs,1937-12-21,Musical,G,5228953251,1937,12,1937Q4,Tuesday,Winter,...,1,1,G,5228953251,5228.953251,0.063622,10923681740,0.132910,26121840571,0.317829
1,Pinocchio,1940-02-09,Adventure,G,2188229052,1940,2,1940Q1,Friday,Winter,...,1,1,G,7417182303,7417.182303,0.026625,32409606385,0.394333,26121840571,0.317829
2,Fantasia,1940-11-13,Musical,G,2187090808,1940,11,1940Q4,Wednesday,Fall,...,2,2,G,9604273111,9604.273111,0.026611,10923681740,0.132910,26121840571,0.317829
3,Song of the South,1946-11-12,Adventure,G,1078510579,1946,11,1946Q4,Tuesday,Fall,...,1,4,G,10682783690,10682.783690,0.013122,32409606385,0.394333,26121840571,0.317829
4,Cinderella,1950-02-15,Drama,G,920608730,1950,2,1950Q1,Wednesday,Winter,...,1,2,G,11603392420,11603.392420,0.011201,8195804484,0.099720,26121840571,0.317829


In [26]:
df["release_quarter"] = "Q" + df["Date Released"].dt.quarter.astype(str)

In [31]:
df.drop(columns=['mpaa_bucket'], inplace=True)

In [32]:
df.to_csv("disney_data.csv", index=False, encoding="utf-8-sig")

In [30]:
genre_mpaa_summary = (df.groupby(['season', 'release_month'], as_index=False)
      .agg(
          total_gross=('Inflation Adjusted Gross', 'sum'),
          avg_gross=('Inflation Adjusted Gross', 'mean'),
          movie_count=('Inflation Adjusted Gross', 'count')
      )
      .sort_values('total_gross', ascending=False)
)

In [29]:
genre_mpaa_summary

,season,release_dow,release_month,total_gross,avg_gross,movie_count
11,Fall,Wednesday,11,7617639583,2.929861e+08,26
24,Summer,Friday,6,6269212486,1.607490e+08,39
14,Spring,Friday,5,5689507629,1.537705e+08,37
2,Fall,Friday,11,5576206397,1.640061e+08,34
46,Winter,Tuesday,12,5288632382,2.644316e+09,2
25,Summer,Friday,7,4647969079,1.549323e+08,30
36,Winter,Friday,12,4427059200,1.526572e+08,29
31,Summer,Wednesday,6,4354530992,3.628776e+08,12
12,Spring,Friday,3,4047251516,7.358639e+07,55
35,Winter,Friday,2,3916873535,1.030756e+08,38
